# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "..\output_data\weather_data.csv"

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=['Unnamed: 0'])
weather_data_df


,City,Cloudiness,County,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,kamenka,88,RU,1586961939,75,51.32,42.77,41.00,10.00
1,kavaratti,7,IN,1586961940,70,10.57,72.64,85.24,7.02
2,hermanus,100,ZA,1586961941,77,-34.42,19.23,66.00,5.86
3,mareeba,75,AU,1586961941,83,-17.00,145.43,77.00,10.29
4,busselton,32,AU,1586961941,55,-33.65,115.33,55.00,6.91
...,...,...,...,...,...,...,...,...,...
554,labuhan,45,ID,1586962080,78,-6.88,112.21,80.82,3.27
555,ginda,40,IN,1586962080,27,30.69,78.49,58.50,6.08
556,mehran,20,IR,1586962025,22,33.12,46.16,69.80,6.93
557,dromolaxia,100,CY,1586962081,46,34.88,33.59,70.00,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key)
locations = weather_data_df[['Lat', 'Lng']]

humidity = weather_data_df['Humidity'].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=5, point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
new_types_df = pd.DataFrame(weather_data_df, columns = ['City', 'Max Temp', 'Wind Speed', 'Cloudiness'])

max_temp = (new_types_df['Max Temp']<= 80) & (new_types_df['Max Temp'] >= 72)
wind_speed = new_types_df['Wind Speed'] < 10
cloudiness = new_types_df['Cloudiness'] == 0

new_types_df[max_temp & wind_speed & cloudiness]

ideal_conditions_df = new_types_df[max_temp & wind_speed & cloudiness].dropna()
ideal_conditions_df['Latitude'] = weather_data_df['Lat']
ideal_conditions_df['Longitude'] = weather_data_df['Lng']
ideal_conditions_df

,City,Max Temp,Wind Speed,Cloudiness,Latitude,Longitude
142,sakakah,78.80,4.70,0,29.97,40.21
196,saint-francois,75.20,6.93,0,46.42,3.91
459,rawah,79.61,9.42,0,34.48,41.91
470,khakhea,78.40,7.96,0,-24.69,23.49


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = pd.DataFrame(ideal_conditions_df)
hotel_df['Hotel Name'] = ""
hotel_df['County'] = weather_data_df['County']
hotel_df

,City,Max Temp,Wind Speed,Cloudiness,Latitude,Longitude,Hotel Name,County
142,sakakah,78.80,4.70,0,29.97,40.21,,SA
196,saint-francois,75.20,6.93,0,46.42,3.91,,FR
459,rawah,79.61,9.42,0,34.48,41.91,,IQ
470,khakhea,78.40,7.96,0,-24.69,23.49,,BW


In [6]:
target_coordinates = f"{hotel_df['Latitude']},{hotel_df['Longitude']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": api_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [7]:
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": api_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [8]:
hotel_name

['Raoum Inn Hotel', "Maison Vin d'Hus", nan, nan]

In [9]:
hotel_df['Hotel Name'] = hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>County</dt><dd>{County}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))